cit.014	Human Settlements

Description: http://ghsl.jrc.ec.europa.eu/ghs_bu.php 
Download as http://ghsl.jrc.ec.europa.eu/datasets.php

GHS_BUILT_LDS1975_GLOBE_R2016A_54009_250 (215MB)
GHS_BUILT_LDS1990_GLOBE_R2016A_3857_38 (854MB) 
GHS_BUILT_LDS2000_GLOBE_R2016A_3857_38 (892MB) 
GHS_BUILT_LDS2014_GLOBE_R2016A_3857_38 (900MB)

File type: Geotiff

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import crs

import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

Define local file locations

In [2]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit_014/"

file_name1 = "GHS_1975_1k"
file_name2 = "GHS_1990_1k"
file_name3 = "GHS_2000_1k"
file_name4 = "GHS_2014_1k"

upload_name1 = "cit_014_1975"
upload_name2 = "cit_014_1990"
upload_name3 = "cit_014_2000"
upload_name4 = "cit_014_2014"

local_orig1 = local_folder + file_name1 + ".tif"
local_orig2 = local_folder + file_name2 + ".tif" 
local_orig3 = local_folder + file_name3 + ".tif"
local_orig4 = local_folder + file_name4 + ".tif"

local_inter1 = local_folder + file_name1 + ".vrt"
local_inter2 = local_folder + file_name2 + ".vrt"
local_inter3 = local_folder + file_name3 + ".vrt"
local_inter4 = local_folder + file_name4 + ".vrt"
 
local_edit1 = local_folder + upload_name1 + "_edit.tif"
local_edit2 = local_folder + upload_name2 + "_edit.tif"
local_edit3 = local_folder + upload_name3 + "_edit.tif"
local_edit4 = local_folder + upload_name4 + "_edit.tif"

s3

In [3]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cit_014_human_settlements/"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [ ]:
band_ids = [upload_name1, upload_name2, upload_name3, upload_name4, upload_name5, upload_name6, upload_name7]
merge_name = "cit_014_human_settlements.tif"
s3_key_merge = s3_folder + merge_name

In [4]:
os.getcwd()
os.chdir(local_folder)
os.environ["local_orig1"] = local_orig1
os.environ["local_edit1"] = local_edit1
os.environ["local_orig2"] = local_orig2
os.environ["local_edit2"] = local_edit2
os.environ["local_orig3"] = local_orig3
os.environ["local_edit3"] = local_edit3
os.environ["local_orig4"] = local_orig4
os.environ["local_edit4"] = local_edit4
os.environ["local_inter1"] = local_inter1
os.environ["local_inter2"] = local_inter2
os.environ["local_inter3"] = local_inter3
os.environ["local_inter4"] = local_inter4


In [ ]:
files = [local_orig1, local_orig2, local_orig3, local_orig4]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

In [5]:
!gdalwarp -overwrite -t_srs epsg:4326 -co compress=lzw -co BLOCKXSIZE=256  -co BLOCKYSIZE=256 -of vrt %local_orig2%  %local_inter2%
!gdalwarp -overwrite -t_srs epsg:4326 -co compress=lzw -co BLOCKXSIZE=256  -co BLOCKYSIZE=256 -of vrt %local_orig3%  %local_inter32%
!gdalwarp -overwrite -t_srs epsg:4326 -co compress=lzw -co BLOCKXSIZE=256  -co BLOCKYSIZE=256 -of vrt %local_orig4%  %local_inter42%


In [ ]:
# !gdal_translate -co compress=LZW  -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -tr 0.000277778 0.000277778 -of GTiff -co TILED=YES %local_orig1%  %local_inter1%
!gdal_translate -co compress=LZW  -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -tr 0.000277778 0.000277778 -of GTiff -co TILED=YES %local_inter2% %local_edit2%
!gdal_translate -co compress=LZW  -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -tr 0.000277778 0.000277778 -of GTiff -co TILED=YES %local_inter3% %local_edit3%
!gdal_translate -co compress=LZW  -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -tr 0.000277778 0.000277778 -of GTiff -co TILED=YES %local_inter4% %local_edit3%


In [ ]:
!gdal_translate -co compress=LZW  -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -tr 0.000277778 0.000277778 -of GTiff -co TILED=YES %local_inter2% %local_edit2%

In [ ]:
gdalwarp -overwrite -t_srs epsg:4326 -co compress=lzw -co BLOCKXSIZE=512 -tr 0.00833333 0.00833333 -co BLOCKYSIZE=512 -of vrt GHS_1990_1k.tif GHS_1990_1k.vrt
gdal_translate -co compress=LZW  -co BLOCKXSIZE=512 -co BLOCKYSIZE=512 -tr 0.00833333 0.00833333 -of GTiff -co TILED=YES GHS_1990_1k.vrt cit_014_1990.tif

In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -of vrt %local_orig1% %local_edit1%

In [ ]:
!gdal_translate -co compress=LZW %local_edit1% %local_edit2% 

In [ ]:
print1 ="gdalwarp -overwrite -t_srs epsg:4326 -co TILED=YES -co BIGTIFF=YES" + " " +local_orig1 +" "+ local_edit1
print(print1)

In [ ]:
local_orig = [local_orig1, local_orig2, local_orig3, local_orig4, local_orig5, local_orig6, local_orig7]
local_edit = [local_edit1, local_edit2, local_edit3, local_edit4, local_edit5, local_edit6, local_edit7]
printlist = "gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw" + " " + str(local_orig)+ " " +str(local_edit)
print(printlist)

In [ ]:
print1 ="rio warp" + " " +local_orig1 +" "+ local_edit1 + " "+"--force-overwrite --dst-crs epsg:4326 --co compress=LZW --co tiled=true --co blockxsize=256 --co blockysize=256"
print(print1)

In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig2% %local_edit2%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig3% %local_edit3%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig4% %local_edit4%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig5% %local_edit5%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig6% %local_edit6%


In [ ]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig7% %local_edit7%

In [ ]:
merge_files = [local_edit1, local_edit2, local_edit3, local_edit4, local_edit5, local_edit6, local_edit7]
tmp_merge = local_folder + merge_name


In [ ]:
gdalwarp -overwrite -t_srs epsg:4326 -co compress=lzw -co BLOCKXSIZE=512 -of vrt GHS_BUILT_LDS1975_GLOBE_R2016A_54009_1k_v1_0.tif GHS_1975_blocksize.vrt
gdal_translate -co compress=LZW -co BLOCKXSIZE=128 -of GTiff -co TILED=YES GHS_1975_blocksize.vrt GHS_1975_blocksize.tif

Use rasterio to reproject and compress

In [ ]:
with rio.open(merge_files[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files)
)
    
with rio.open(tmp_merge, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)


Upload orig and edit files to s3

In [ ]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig5, s3_bucket, s3_key_orig5,
                         Callback=ProgressPercentage(local_orig5))
s3_upload.upload_file(local_orig7, s3_bucket, s3_key_orig7,
                         Callback=ProgressPercentage(local_orig7))
s3_upload.upload_file(local_orig9, s3_bucket, s3_key_orig9,
                         Callback=ProgressPercentage(local_orig9))
s3_upload.upload_file(local_orig11, s3_bucket, s3_key_orig11,
                         Callback=ProgressPercentage(local_orig11))
s3_upload.upload_file(local_orig13, s3_bucket, s3_key_orig13,
                         Callback=ProgressPercentage(local_orig13))


# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit5, s3_bucket, s3_key_edit5,
                         Callback=ProgressPercentage(local_edit5))
s3_upload.upload_file(local_edit7, s3_bucket, s3_key_edit7,
                         Callback=ProgressPercentage(local_edit7))
s3_upload.upload_file(local_edit9, s3_bucket, s3_key_edit9,
                         Callback=ProgressPercentage(local_edit9))
s3_upload.upload_file(local_edit11, s3_bucket, s3_key_edit11,
                         Callback=ProgressPercentage(local_edit11))
s3_upload.upload_file(local_edit13, s3_bucket, s3_key_edit13,
                         Callback=ProgressPercentage(local_edit13))

# Merged 

s3_upload.upload_file(tmp_merge, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(tmp_merge))

In [ ]:
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_merge
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_merge


In [ ]:
!gsutil cp %Zs3_key% %Zgs_key%


In [ ]:
os.environ["asset_id"] = "users/resourcewatch/cit_014_human_settlements"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%


In [ ]:
os.environ["band_names"] = str(band_ids)
!earthengine asset set -p band_names="%band_names%" %asset_id%
